<h1>Reccomendation System</h1>

<h2>Importing Libraries</h2>

In [2]:
import numpy as np
import random as rn
import pandas as pd
import seaborn as sns
import re
import math

<h2>Importing Database</h2>

In [3]:
apps = pd.read_csv(r"googleplaystore.csv")

<h1>Data Cleanup</h1>

In [4]:
apps = apps.drop(columns=['Type', 'Price', 'Genres', 'Last Updated', 'Current Ver'])

In [5]:
apps['Android Ver'] = apps['Android Ver'].str.replace('[^\d.]', '')
avgVer = pd.to_numeric(apps['Android Ver'], errors='coerce').mean()
apps['Android Ver'] = apps['Android Ver'].replace('', avgVer)
apps['Android Ver'] = apps['Android Ver'].str.replace('.','', regex=True).str.replace(',', '.', regex=True).astype(float)
apps['Size'] = apps['Size'].str.replace('[^\d.]', '')
apps['Installs'] = apps['Installs'].str.replace('[^\d.]', '')

avgSize = pd.to_numeric(apps['Size'], errors='coerce').mean()
apps['Size'] = apps['Size'].replace('', avgSize)
apps = apps.dropna()
apps = apps.astype({'Rating':'float', 'Reviews':'float', 'Size':'float', 'Installs':'float', 'Android Ver':'float'})
apps

C:\Users\elere\AppData\Local\Temp\ipykernel_28008\3166082878.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  apps['Android Ver'] = apps['Android Ver'].str.replace('[^\d.]', '')
C:\Users\elere\AppData\Local\Temp\ipykernel_28008\3166082878.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  apps['Size'] = apps['Size'].str.replace('[^\d.]', '')
C:\Users\elere\AppData\Local\Temp\ipykernel_28008\3166082878.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  apps['Installs'] = apps['Installs'].str.replace('[^\d.]', '')


,App,Category,Rating,Reviews,Size,Installs,Content Rating,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159.0,19.0,10000.0,Everyone,403.0
1,Coloring book moana,ART_AND_DESIGN,3.9,967.0,14.0,500000.0,Everyone,403.0
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510.0,8.7,5000000.0,Everyone,403.0
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644.0,25.0,50000000.0,Teen,42.0
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967.0,2.8,100000.0,Everyone,44.0
...,...,...,...,...,...,...,...,...
10832,FR Tides,WEATHER,3.8,1195.0,582.0,100000.0,Everyone,21.0
10833,Chemin (fr),BOOKS_AND_REFERENCE,4.8,44.0,619.0,1000.0,Everyone,22.0
10834,FR Calculator,FAMILY,4.0,7.0,2.6,500.0,Everyone,41.0
10836,Sya9a Maroc - FR,FAMILY,4.5,38.0,53.0,5000.0,Everyone,41.0


In [6]:
apps.sort_values(by=['Android Ver'])

,App,Category,Rating,Reviews,Size,Installs,Content Rating,Android Ver
9212,Tamilnadu Electricity Info,TOOLS,3.6,216.0,811.0,10000.0,Everyone,10.0
8919,Bible du Semeur-BDS (French),BOOKS_AND_REFERENCE,4.5,313.0,6.9,50000.0,Everyone,10.0
200,SuperLivePro,BUSINESS,4.3,46353.0,21.0,1000000.0,Everyone,15.0
9566,Trazado de tuberia El Tubero,TOOLS,4.5,313.0,4.4,10000.0,Everyone,15.0
1542,Wifi Connect Library,LIBRARIES_AND_DEMO,3.9,58055.0,41.0,5000000.0,Everyone,15.0
...,...,...,...,...,...,...,...,...
3474,ASUS Quick Memo,PRODUCTIVITY,4.5,23089.0,1.6,10000000.0,Everyone,5080.0
4151,Google Now Launcher,TOOLS,4.2,857215.0,7.9,100000000.0,Everyone,41711.0
1572,ZenUI Safeguard,LIFESTYLE,4.5,100.0,7.1,1000000.0,Everyone,70711.0
352,Browser 4G,COMMUNICATION,4.3,192948.0,6.6,10000000.0,Everyone,403711.0


In [7]:
class DataProcessing:
    @staticmethod
    def shuffle(x):
        for i in range(len(x)-1,0,-1):
            j=rn.randint(0,i-1)
            x.iloc[j], x.iloc[i] = x.iloc[i], x.iloc[j]
            
    @staticmethod
    def normalization(x):
        values=x.select_dtypes(exclude="object")
        columnNames=values.columns.tolist()
        for column in columnNames[:-1]:
            data = x.loc[:,column]
            max1=max(data)
            min1=min(data)
            for row in range(0,len(x),1):
                xprim=(x.at[row, column]-min1)/(max1-min1)
                x.at[row, column]=xprim
                
    @staticmethod
    def split(x,k):
        return x[:int(len(x)*k//1)], x[int(len(x)*k//1 +1):]

In [8]:
DataProcessing.shuffle(apps)

In [9]:
trainingSet, validatingSet = DataProcessing.split(apps, 0.95)
validatingSet = validatingSet.reset_index(drop=True)
DataProcessing.normalization(validatingSet)
validatingSet.sort_values(by=['Size'])
trainingSet = trainingSet.reset_index(drop=True)
DataProcessing.normalization(trainingSet)

<h2>k-nearest neighbors algorithm</h2>

In [10]:
class KNN:
    @staticmethod
    def distance(v1, v2):
        return math.sqrt(math.pow(v1[2] - v2[2], 2) + math.pow(v1[3] - v2[3], 2) + math.pow(v1[4] - v2[4], 2) + math.pow(v1[5] - v2[5], 2))
    def clustering(dataset, sample):
        distances = []
        for i in range(len(dataset)):
            distances.append(KNN.distance(sample, dataset.iloc[i]))
            if(dataset.iloc[i]['App'] == 'CZ-Help'):
                print(KNN.distance(sample, dataset.iloc[i]))
        dataset["distance"] = distances
        dataset = dataset.sort_values("distance")
        
        return dataset.head(10)

<h2>Apps Recommendation</h2>

In [11]:
avgRating = pd.to_numeric(validatingSet['Rating'], errors='coerce').mean()
avgReviews = pd.to_numeric(validatingSet['Reviews'], errors='coerce').mean()
avgSize = pd.to_numeric(validatingSet['Size'], errors='coerce').mean()
avgInstalls = pd.to_numeric(validatingSet['Installs'], errors='coerce').mean()
avgApp = pd.DataFrame(columns=['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Content Rating', 'Android Ver'])
avgApp.loc[0] = ['a', 'b', avgRating, avgReviews, avgSize, avgInstalls, 'c', 'd']
newdf = trainingSet.reset_index(drop=True)
recommendedApps = KNN.clustering(newdf[:100], avgApp.loc[0])
recommendedApps

C:\Users\elere\AppData\Local\Temp\ipykernel_28008\3474841398.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["distance"] = distances


,App,Category,Rating,Reviews,Size,Installs,Content Rating,Android Ver,distance
74,Weather Crave,WEATHER,0.800,0.002970,0.043303,0.0050,Everyone,41.0,0.017790
68,Cook Baked Lasagna,FAMILY,0.800,0.001253,0.022155,0.0100,Everyone,40.0,0.019658
28,"Talkatone: Free Texts, Calls & Phone Number",COMMUNICATION,0.775,0.002941,0.024169,0.0100,Everyone,42.0,0.026269
48,Free Dating App - Meet Local Singles - Flirt Chat,DATING,0.775,0.000018,0.036395,0.0001,Mature 17+,40.0,0.028608
13,Heart Surgery Emergency Doctor,FAMILY,0.775,0.000050,0.028197,0.0001,Teen,40.0,0.029208
92,"Shopkick: Free Gift Cards, Shop Rewards & Deals",SHOPPING,0.825,0.004761,0.042296,0.0100,Everyone,44.0,0.032594
21,Glowing Flowers Live Wallpaper,PERSONALIZATION,0.800,0.001486,0.006042,0.0050,Everyone,41.0,0.032963
33,K-9 Mail,COMMUNICATION,0.800,0.001970,0.004129,0.0050,Everyone,403.0,0.034432
81,Undead Assault,GAME,0.825,0.000105,0.036254,0.0001,Everyone 10+,23.0,0.035713
77,Baby Care & Tracker,PARENTING,0.775,0.000007,0.010070,0.0001,Everyone,403.0,0.037557


<h2>KD Tree Algorithm</h2>

In [42]:
class Node:
    def __init__(self, point, left=None, right=None, name=None):
        self.point = point
        self.left = left
        self.right = right
        self.name = name


class KDTree:
    def __init__(self, points):
        self.k = 4  # Number of dimensions
        self.root = self.build_tree(points)

    def build_tree(self, points, depth=0):
        if len(points) == 0:
            return None

        # Select axis based on depth
        axis = depth % self.k

        # Sort points based on the axis
        sorted_points = points.sort_values(by=points.columns[axis])

        # Find the median
        median_index = len(sorted_points) // 2
        median = sorted_points.iloc[median_index]

        # Recursively build subtrees
        left_points = sorted_points[:median_index]
        right_points = sorted_points[median_index + 1:]

        left = self.build_tree(left_points, depth + 1)
        right = self.build_tree(right_points, depth + 1)
        return Node(median[2:6], left, right, median[0])

    def find_nearest_neighbors(self, query_point, k=1):
        nearest_points = []
        distances = []
        nearest_names = []

        def search(node, depth=0):
            if node is None:
                return

            # Select axis based on depth
            axis = depth % self.k

            # Traverse left or right subtree based on query point
            if query_point[axis] < node.point[axis]:
                search(node.left, depth + 1)
            else:
                search(node.right, depth + 1)

            # Calculate distance between query point and current point
            distance = np.linalg.norm(np.array(query_point) - np.array(node.point))
            if(node.name == 'Weather Crave'):
                print("weather")
            # If the number of nearest points is less than k, add current point
            if len(nearest_points) < k:
                nearest_points.append(node.point)
                distances.append(distance)
                nearest_names.append(node.name)
            else:
                # If the current point is closer than the farthest nearest point,
                # replace the farthest point with the current point
                max_distance_index = np.argmax(distances)
                if distance < distances[max_distance_index]:
                    #print(node.name)
                    nearest_points[max_distance_index] = node.point
                    distances[max_distance_index] = distance
                    nearest_names[max_distance_index] = node.name

            # Check if there can be closer points on the other side of the splitting axis
            if abs(query_point[axis] - node.point[axis]) < distances[np.argmax(distances)] or len(nearest_points) < k:
                if query_point[axis] < node.point[axis]:
                    search(node.right, depth + 1)
                else:
                    search(node.left, depth + 1)
        search(self.root)
        return nearest_names

In [43]:
#numpyApps = trainingSet[:30][['Rating', 'Reviews', 'Size', 'Installs']].to_numpy() # Convert DataFrame to NumPy array

kdtree = KDTree(newdf[:100])

query_point = avgApp[['Rating', 'Reviews', 'Size', 'Installs']].to_numpy()[0]
print("Query point:", query_point)
nearest_neighbors = kdtree.find_nearest_neighbors(query_point, k=5)

print("Nearest neighbors:")
for neighbor in nearest_neighbors:
    print(neighbor)
#trainingSet.iloc[4073]
# print(newdf.iloc[12])
# print(KNN.distance(trainingSet.iloc[74], avgApp.loc[0]))
# print(KNN.distance(trainingSet.iloc[77], avgApp.loc[0]))
# print(kdtree.root.left.left.left.left.left.left.point)

Query point: [0.79539801 0.01564703 0.03447223 0.01252508]
Nearest neighbors:
Do it (Tomorrow)
Cook Baked Lasagna
F-Gas Solutions
Free Dating App - Meet Local Singles - Flirt Chat
Baby Care & Tracker


In [38]:
trainingSet.loc[trainingSet['App'] == 'Baby Care & Tracker']

,App,Category,Rating,Reviews,Size,Installs,Content Rating,Android Ver
77,Baby Care & Tracker,PARENTING,0.775,0.000007,0.01007,0.0001,Everyone,403.0
